# Variational MPS and iTEBD for Three-body Spin Model

This notebook contains code for two different approaches to obtain ground state of a three-body spin model, namely:
1. variational matrix product state method (variational MPS),
2. infinite-time evolving block decimation method (iTEBD).

The model we are interested in is of the following Hamiltonian:
$$
H = -\sum_j (\sigma_{j-1}^z \sigma_{j}^x \sigma_{j+1}^z + \sigma_j^y).
$$
And we would like to investigate the ground state of this model on a $N=10$ lattice using 2-site variational MPS method and on an infinite length lattice using 3-site iTEBD method.

# Variational MPS

Firstly, let's give variational MPS a try.

## MPO form of Hamiltonian

To implement variational MPS, we shall construct the matrix product operator (MPO) first.

The MPO for this model is
$$
M = \begin{pmatrix}
I &\sigma^z &0 &\sigma^y\\
0 &0 &\sigma^x &0\\
0 &0 &0 &\sigma^z\\
0 &0 &0 &I
\end{pmatrix}.
$$
Let's implement this.

In [1]:
import numpy as np

Sx = np.array([[0, 1], [1, 0]])
Sy = np.array([[0, -1j], [1j, 0]])
Sz = np.array([[1, 0], [0, -1]])
I = np.eye(2)

mpo = np.zeros((4, 2, 4, 2), dtype=complex)
mpo[0, :, 0, :] = I
mpo[0, :, 1, :] = Sz
mpo[0, :, 3, :] = Sy
mpo[1, :, 2, :] = Sx
mpo[2, :, 3, :] = Sz
mpo[3, :, 3, :] = I

## State Initialization

The second thing we shall do is to define a matrix product state and initialize it.

In [2]:
# import subroutines for tensor manipulation
import sys
sys.path.append("..")
from sample_code.vMPS_iTEBD import Sub180221 as Sub

def init_mps(n_site, dim_phys, dim_bond):
    mps = [None] * n_site
    for i in range(n_site):
        dim_left = min(dim_phys ** i, dim_phys ** (n_site - i), dim_bond)
        dim_right = min(dim_phys ** (i+1), dim_phys ** (n_site-1-i), dim_bond)
        mps[i] = np.random.rand(dim_left, dim_phys, dim_right)
    
    # canonicalize
    U = np.eye(np.shape(mps[-1])[-1])
    for i in range(n_site - 1, 0, -1):
        U, mps[i] = Sub.Mps_LQP(mps[i], U)
    
    return mps

## Environment Initialization

The third step is to multiply out all the MPOs and sandwitch it with the MPS to obtain the whole tensor.

This step is called environment initialization (i.e. all the tensors in $\bra{\psi}H\ket{\psi}$).

In [3]:
def initH(mpo, mps):
    n_site = len(mps)
    dim_mpo = np.shape(mpo)[0]
    
    H_left = [None] * n_site
    H_right = [None] * n_site
    
    H_left[0] = np.zeros((1, dim_mpo, 1))
    H_left[0][0, 0, 0] = 1
    H_right[-1] = np.zeros((1, dim_mpo, 1))
    H_right[-1][0, -1, 0] = 1
    
    for i in range(n_site - 1, 0, -1):
        H_right[i - 1] = Sub.NCon([H_right[i], mps[i], mpo, np.conj(mps[i])], 
        [[1, 3, 5], [-1, 2, 1], [-2, 2, 3, 4], [-3, 4, 5]])
    
    return H_left, H_right

## Site Update

The final step is to update each two site in an iterative manner and obtain the ground state in covergence.

We implement the two site update precedure first and then carry out the sweeping.

For the two site update precedure, we can generalize it to **arbitrary n sites** update precedure as below.

In [23]:
import scipy.sparse.linalg as LAs

def update_sites(mpo, H_left, H_right, site_tensor_list):
    '''
    Update arbitrary n consecutive sites in the MPS
    '''
    shape_site_tensor_list = list(map(np.shape, site_tensor_list))
    dim_site_tensor_list = list(map(lambda x: x[1], shape_site_tensor_list))
    n_sites = len(shape_site_tensor_list)
    print(shape_site_tensor_list)
    
    contract_rule = [[-1, 1, -(n_sites + 3)]] + \
                    [[i, -(n_sites + 3 + i), i + 1, -(1 + i)] for i in range(1, n_sites + 1)] + \
                    [[-(2 * n_sites + 4), n_sites + 1, -(n_sites + 2)]]
    H_eff = Sub.NCon([H_left] + [mpo] * n_sites + [H_right], contract_rule)
    print(H_eff.shape)
    H_eff = Sub.Group(H_eff, [[i for i in range(n_sites + 2)], [i for i in range(n_sites + 2, 2 * n_sites + 4)]])
    print(H_eff.shape)
    eigval, eigvec = LAs.eigsh(H_eff, k=1, which='SA')

    updated_site_tensor = np.reshape(eigvec, [H_left.shape[-1]] + dim_site_tensor_list + [H_right.shape[0]])
    print(updated_site_tensor.shape)

    updated_site_tensor_list = [None] * n_sites
    for i in range(n_sites - 1):
        svd_tensor = Sub.Group(updated_site_tensor, [[0, 1], list(range(2, len(updated_site_tensor.shape)))])
        print(svd_tensor.shape)
        u, s, v = np.linalg.svd(svd_tensor, full_matrices=False)
        print(u.shape, s.shape, v.shape)
        updated_site_tensor_list[i] = u.reshape(shape_site_tensor_list[i])
        updated_site_tensor = np.diag(s) @ v
    updated_site_tensor_list[-1] = updated_site_tensor.reshape(shape_site_tensor_list[-1])
    
    return updated_site_tensor_list, eigval

Now we can implement the sweeping precedure.

Here we update two sites per step.

In [24]:
def sweep(mpo, H_left, H_right, mps):
    n_sites = len(mps)
    eig0 = np.zeros(n_sites)
    eig1 = np.zeros(n_sites)
    
    for r in range(100):
        print(f'Iteration {r}:')
    
        for i in range(n_sites - 1):
            mps[i:i+2], eig1[i] = update_sites(mpo, H_left[i], H_right[i+1], mps[i:i+2])
            mps[i], U = Sub.Mps_QR0P(mps[i])
            H_left[i+1] = Sub.NCon([H_left[i], np.conj(mps[i]), mpo, mps[i]], 
                            [[1, 3, 5], [1, 2, -1], [3, 4, -2, 2], [5, 4, -3]])
            mps[i+1] = np.tensordot(U, mps[i+1], (1, 0))
        
        for i in range(n_sites - 1, 0, -1):
            mps[i-2:i], eig1[i] = update_sites(mpo, H_left[i-1], H_right[i], mps[i-2:i])
            U, mps[i] = Sub.Mps_LQ0P(mps[i])
            H_right[i-1] = Sub.NCon([H_right[i], mps[i], mpo, np.conj(mps[i])], 
                            [[1, 3, 5], [-1, 2, 1], [-2, 2, 3, 4], [-3, 4, 5]])
            mps[i-1] = np.tensordot(mps[i-1], U, (2,0))
        
        print(eig1)
        if abs(eig1[1]-eig0[1]) < 1.0e-7:
            break
        eig0 = eig1.copy()
    
    print(f'energy per site: {eig1/n_sites}')
    
    return mps

## Demo

Now let's demonstrate this approach on our model.

In [25]:
n_sites = 10; dim_phys = 2; dim_bond = 4
mpo = mpo
mps = init_mps(n_sites, dim_phys, dim_bond)
H_left, H_right = initH(mpo, mps)
mps = sweep(mpo, H_left, H_right, mps)

Iteration 0:
[(1, 2, 2), (2, 2, 4)]
(1, 2, 2, 4, 1, 2, 2, 4)
(16, 16)
(1, 2, 2, 4)
(2, 8)
(2, 2) (2,) (2, 8)
[(2, 2, 4), (4, 2, 4)]
(2, 2, 2, 4, 2, 2, 2, 4)
(32, 32)
(2, 2, 2, 4)
(4, 8)
(4, 4) (4,) (4, 8)
[(4, 2, 4), (4, 2, 4)]
(4, 2, 2, 4, 4, 2, 2, 4)
(64, 64)
(4, 2, 2, 4)
(8, 8)
(8, 8) (8,) (8, 8)


ValueError: cannot reshape array of size 64 into shape (4,2,4)